In [1]:
import random, math
class Node:
    def __init__(self, state, parent=None):
        self.state = state  # Current game state
        self.parent = parent  # Parent node
        self.children = []  # Child nodes
        self.visits = 0  # Number of visits
        self.wins = 0  # Number of wins

    def is_leaf(self):
        return len(self.children) == 0

    def is_terminal(self):
        return self.state.is_game_over()

    def expand(self):
        if not self.is_terminal():
            possible_moves = self.state.get_possible_moves()
            for move in possible_moves:
                new_state = self.state.copy()
                new_state.make_move(move)
                self.children.append(Node(new_state, parent=self))

    def select_child(self, exploration_weight=1.41):
        max_ucb = float("-inf")
        selected_child = None
        for child in self.children:
            if child.visits == 0:
                return child
            ucb = child.wins / child.visits + exploration_weight * math.sqrt(
                math.log(self.visits) / child.visits
            )
            if ucb > max_ucb:
                max_ucb = ucb
                selected_child = child
        return selected_child

    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(result)


class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9
        self.current_player = 'X'

    def copy(self):
        new_board = TicTacToe()
        new_board.board = self.board[:]
        new_board.current_player = self.current_player
        return new_board

    def make_move(self, move):
        if self.board[move] == ' ':  # Check if the spot is empty
            self.board[move] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
        else:
            raise ValueError("Upss, wrong choose.")

    def is_game_over(self):
        # Check rows, columns, and diagonals
        for i in range(3):
            if self.board[i] == self.board[i + 3] == self.board[i + 6] != ' ':
                return True
            if self.board[3 * i] == self.board[3 * i + 1] == self.board[3 * i + 2] != ' ':
                return True
        if self.board[0] == self.board[4] == self.board[8] != ' ':
            return True
        if self.board[2] == self.board[4] == self.board[6] != ' ':
            return True
        # Check for draw
        if ' ' not in self.board:
            return True
        return False

    def get_possible_moves(self):
        return [i for i in range(9) if self.board[i] == ' ']

    def get_winner(self):
        # Check rows, columns, and diagonals
        for i in range(3):
            if self.board[i] == self.board[i + 3] == self.board[i + 6] != ' ':
                return self.board[i]
            if self.board[3 * i] == self.board[3 * i + 1] == self.board[3 * i + 2] != ' ':
                return self.board[3 * i]
        if self.board[0] == self.board[4] == self.board[8] != ' ':
            return self.board[0]
        if self.board[2] == self.board[4] == self.board[6] != ' ':
            return self.board[2]
        return None

    def print_board(self):
        for i in range(3):
            print(' | '.join(self.board[i * 3:i * 3 + 3]))
            if i < 2:
                print('-' * 5)

def uct_search(root_state, num_iterations):
    root_node = Node(root_state)
    for _ in range(num_iterations):
        node = root_node
        # Selection
        while not node.is_leaf() and not node.is_terminal():
            node = node.select_child()
        # Expansion
        if not node.is_terminal():
            node.expand()
            if node.children:  
                node = random.choice(node.children)
            else:  #No moves left to play 
                return None
        # Simulation
        simulation_state = node.state.copy()
        while not simulation_state.is_game_over():
            possible_moves = simulation_state.get_possible_moves()
            if possible_moves:
                random_move = random.choice(possible_moves)
                simulation_state.make_move(random_move)
            else:  #No moves left to play
                break
        # Backpropagation
        winner = simulation_state.get_winner()
        result = 1 if winner == 'X' else 0 if winner == 'O' else 0.5
        node.backpropagate(result)

    if root_node.children:
        best_move = max(root_node.children, key=lambda child: child.visits).state.get_possible_moves()
        if best_move:  # Check if there are any moves left
            return best_move[0]
    return None  



if __name__ == "__main__":
    game = TicTacToe()
    while not game.is_game_over():
        game.print_board()
        if game.current_player == 'X':
            move = uct_search(game, 1000)
            if move is None:
                print("The END")
                break  
            print("Terminator turn:", move)
        else:
            valid_move = False
            while not valid_move:
                try:
                    move = int(input("Your move (0-8): "))
                    if game.board[move] == ' ':
                        valid_move = True
                    else:
                        print("Do you have eyes?. Choose another.")
                except (ValueError, IndexError):
                    print("Eres tonto o que. Enter a number between 0 and 8.")
        game.make_move(move)
    game.print_board()
    winner = game.get_winner()
    if winner:
        print(f"{winner} wins!")
    else:
        print("Drawwwwwwwwwwwwwwwww!")

  |   |  
-----
  |   |  
-----
  |   |  
Terminator turn: 0
X |   |  
-----
  |   |  
-----
  |   |  
Your move (0-8): 4
X |   |  
-----
  | O |  
-----
  |   |  
Terminator turn: 1
X | X |  
-----
  | O |  
-----
  |   |  
Your move (0-8): 7
X | X |  
-----
  | O |  
-----
  | O |  
Terminator turn: 3
X | X |  
-----
X | O |  
-----
  | O |  
Your move (0-8): 8
X | X |  
-----
X | O |  
-----
  | O | O
Terminator turn: 5
X | X |  
-----
X | O | X
-----
  | O | O
Your move (0-8): 2
X | X | O
-----
X | O | X
-----
  | O | O
The END
X | X | O
-----
X | O | X
-----
  | O | O
Drawwwwwwwwwwwwwwwww


As we know, based on a simulation policy, typically a simple random	policy,	continue playing from the expansion	node until a terminal node.
In our MCTS algorithm, the roll-out policy refers to how the game is played out from a node until a result (win, lose, or draw) is achieved. Our implementation is kinda simple, becuase we have chosen to randomly select moves until the game is ove
The oponent policy is also straighforward, just select random moves from the available moves until the game ends.
We have chose this approach because it ensures that the simulation phase explores a wide variety of game outcomes without any bias towards specific moves or strategies.

For the selection policy, we have selected something a bit more complex that we have seen in the lecture, we use the upper confidence bound(UCB) because is a very popular heuristic.
For those who didn't know how it works, the upper confidence bound works like a trade-off, it try to balance between exploring options and choosing the best options, this method is very good when apply to games, that's why is very popular, here we have some mathematical foundation of the heuristic:

In [ ]:
UCB = Q(v) + c * sqrt(ln(N(v_parent)) / N(v))

Where Q(v): the estimate value of the node v, usually calculate with the average of the simulation results starting in v.

c : is a parameter that control the balance between exploration and exploitation(choosing the best option),  as "c" increases, the search strategy becomes more oriented towards exploration, which can be beneficial in scenarios where actively exploring the search space for new solutions or strategies is important. However, caution should be exercised when selecting an appropriate value for "c", as a very high value may lead to excessive exploration and suboptimal performance in exploiting promising nodes.

N(v): number of times 'v' has been visited.

N(v_parent): number of time parent of 'v' has been visited.


Overall,  the UCB heuristic is used to select child nodes during the selection phase. It guides the search towards promising nodes while ensuring sufficient exploration of the search space.

As we know, the backpropagation works once we know the value of the terminal node, we backpropate it throght the tree, with that we update our idea of how good moves were and the statistics of nodes like N(v) and Q(v).

To continue, we can take a look at how our MCTS works in deep: 
In the simulation phase after selecting a node during the selection phase of MCTS, the algorithm proceeds to simulate the game from that node by making random moves until a terminal state is reached, then within this simulation phase, the code iterates through the game state, randomly selecting moves until the game reaches a terminal state. This is called sampling and in this case its function allows the algorithm to explore the game tree by simulating different sequences of moves from the selected node. By randomly sampling moves, MCTS can gather statistics about the quality of different moves and use this information to guide its decision-making process. 

After the simulation phase, the result of the simulated game is backpropagated up the tree, updating the visit counts and win counts of the nodes along the path from the simulated node to the root. This is done to reflect the outcome of the simulated game in the statistics maintained by each node.

The algorithm works mediocre at best because the algorithm doesn't look for winning, it tries to draw every game. Also, is very fast but this is due to our random selection policies and that is a small and simple game, but this type of algorithm when scaling to some 5x5 grid for example, its computational cost and grows exponentially limiting the scalability to small instances, also, as the number of instances go up the algorithm slows down.

Here we have the improved model with the specifications of the report:

In [5]:
import random
import math


class Node:
    def __init__(self, state, parent=None, move=None):
        self.state = state  # Current game state
        self.parent = parent  # Parent node
        self.move = move  # The move that was made to reach this node
        self.children = []  # Child nodes
        self.visits = 0  # Number of visits
        self.wins = 0  # Number of wins

    def is_leaf(self):
        return len(self.children) == 0

    def is_terminal(self):
        return self.state.is_game_over()

    def expand(self):
        if not self.is_terminal():
            possible_moves = self.state.get_possible_moves()
            for move in possible_moves:
                new_state = self.state.copy()
                new_state.make_move(move)
                self.children.append(Node(new_state, parent=self, move=move))

    def select_child_ucb1(self, exploration_weight=1.41):
        best_value = float("-inf")
        best_child = None
        for child in self.children:
            # Calculate the UCB1 value
            if child.visits > 0:
                win_rate = child.wins / child.visits
                exploration_value = exploration_weight * math.sqrt(math.log(self.visits) / child.visits)
                ucb1 = win_rate + exploration_value
            else:
                ucb1 = float('inf')  

            if ucb1 > best_value:
                best_value = ucb1
                best_child = child
        return best_child

    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(result)


class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9
        self.current_player = 'X'

    def copy(self):
        new_board = TicTacToe()
        new_board.board = self.board[:]
        new_board.current_player = self.current_player
        return new_board

    def make_move(self, move):
        if self.board[move] == ' ':
            self.board[move] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
            return True  # Move successfully made
        else:
            return False  # Move not allowed

    def is_game_over(self):
        # Check rows, columns, and diagonals
        for i in range(3):
            if self.board[i] == self.board[i + 3] == self.board[i + 6] != ' ':
                return True
            if self.board[3 * i] == self.board[3 * i + 1] == self.board[3 * i + 2] != ' ':
                return True
        if self.board[0] == self.board[4] == self.board[8] != ' ':
            return True
        if self.board[2] == self.board[4] == self.board[6] != ' ':
            return True
        # Check for draw
        if ' ' not in self.board:
            return True
        return False

    def get_possible_moves(self):
        return [i for i in range(9) if self.board[i] == ' ']

    def get_winner(self):
        # Check rows, columns, and diagonals
        for i in range(3):
            if self.board[i] == self.board[i + 3] == self.board[i + 6] != ' ':
                return self.board[i]
            if self.board[3 * i] == self.board[3 * i + 1] == self.board[3 * i + 2] != ' ':
                return self.board[3 * i]
        if self.board[0] == self.board[4] == self.board[8] != ' ':
            return self.board[0]
        if self.board[2] == self.board[4] == self.board[6] != ' ':
            return self.board[2]
        return None

    def print_board(self):
        for i in range(3):
            print(' | '.join(self.board[i * 3:i * 3 + 3]))
            if i < 2:
                print('-' * 5)

def uct_search(root_state, num_iterations, exploration_constant=math.sqrt(2)):
    root_node = Node(root_state)
    
    for _ in range(num_iterations):
        node = root_node
        # Selection
        while not node.is_leaf() and not node.is_terminal():
            node = node.select_child_ucb1(exploration_constant)
        
        # Expansion
        if not node.is_terminal():
            node.expand()
            if node.children:
                node = random.choice(node.children)
            else:
                return None  # No possible moves left
        
        # Simulation
        simulation_state = node.state.copy()
        while not simulation_state.is_game_over():
            possible_moves = simulation_state.get_possible_moves()
            if not possible_moves:
                break
            random_move = random.choice(possible_moves)
            simulation_state.make_move(random_move)
        
        # Backpropagation
        winner = simulation_state.get_winner()
        result = 1 if winner == root_state.current_player else 0  # Adjust based on the perspective of the starting player
        node.backpropagate(result)
    
    # Choose the best move
    if root_node.children:
        best_move_node = max(root_node.children, key=lambda child: child.wins / child.visits if child.visits > 0 else 0)
        return best_move_node.move  # Return the move associated with the best node
    
    return None  # No possible moves left


if __name__ == "__main__":
    game = TicTacToe()
    
    # Randomly choose the starting position for the AI
    ai_start_position = random.choice(game.get_possible_moves())
    game.make_move(ai_start_position)
    print("Terminator starts:", ai_start_position)
    
    while not game.is_game_over():
        game.print_board()
        if game.current_player == 'X':
            move = uct_search(game, 10000)
            if move is None:
                print("How can be you so bad?")
                break
            print("Terminator moves:", move)
        else:
            valid_move = False
            while not valid_move:
                try:
                    move = int(input("Your move (0-8): "))
                    if game.board[move] == ' ':
                        valid_move = True
                    else:
                        print("Are you blind maybe? Choose another spot.")
                except (ValueError, IndexError):
                    print("Learn to read thanks. Please enter a number between 0 and 8.")
        game.make_move(move)
    game.print_board()
    winner = game.get_winner()
    if winner:
        print(f"{winner} wins!")
    else:
        print("It's a draw! HAhaahahhaha")

Terminator starts: 6
  |   |  
-----
  |   |  
-----
X |   |  
Your move (0-8): 4
  |   |  
-----
  | O |  
-----
X |   |  
Terminator moves: 3
  |   |  
-----
X | O |  
-----
X |   |  
Your move (0-8): 0
O |   |  
-----
X | O |  
-----
X |   |  
Terminator moves: 8
O |   |  
-----
X | O |  
-----
X |   | X
Your move (0-8): 7
O |   |  
-----
X | O |  
-----
X | O | X
Terminator moves: 1
O | X |  
-----
X | O |  
-----
X | O | X
Your move (0-8): 2
O | X | O
-----
X | O |  
-----
X | O | X
Terminator moves: 5
O | X | O
-----
X | O | X
-----
X | O | X
It's a draw! HAhaahahhaha
